In [3]:
import pandas as pd
import numpy as np
import surprise

In [4]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate


## Loading the 100k dataset from the scikit-surprise library

In [5]:
ls

 Volume in drive D is DATA
 Volume Serial Number is 0CAE-86E8

 Directory of D:\Strawhats\Model\project\movie-recommend\Movie-Recommend\movie2

13-10-2022  14:11    <DIR>          .
13-10-2022  16:22    <DIR>          ..
13-10-2022  11:16    <DIR>          .ipynb_checkpoints
13-10-2022  11:15           197,979 links.csv
27-09-2018  02:20    <DIR>          ml-latest-small
13-10-2022  11:26           978,202 ml-latest-small.zip
13-10-2022  11:11           993,939 movielens.zip
13-10-2022  14:11            31,193 Movie-recommender-2.ipynb
13-10-2022  11:15           494,431 movies.csv
13-10-2022  11:15         2,483,723 ratings.csv
13-10-2022  11:15             8,342 README.txt
13-10-2022  11:15           118,660 tags.csv
               8 File(s)      5,306,469 bytes
               4 Dir(s)  287,591,030,784 bytes free


In [6]:
movies=pd.read_csv('movies.csv')
ratings=pd.read_csv('ratings.csv')
links=pd.read_csv('links.csv')
tags=pd.read_csv('tags.csv')

In [7]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [10]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [11]:
def splitting(text):
    text=text.split('|')
    return text

In [12]:
movies['genres']=movies['genres'].apply(splitting)

In [13]:
movies['imdbId']=links['imdbId']
movies['tmdbId']=links['tmdbId']

In [14]:
movies=movies.dropna()

In [15]:
movies.isna().sum()

movieId    0
title      0
genres     0
imdbId     0
tmdbId     0
dtype: int64

In [16]:
tags.loc[tags['userId']==2]

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
5,2,89774,Tom Hardy,1445715205
6,2,106782,drugs,1445715054
7,2,106782,Leonardo DiCaprio,1445715051
8,2,106782,Martin Scorsese,1445715056


In [17]:
ratings=ratings[['userId','movieId','rating']]

In [18]:
tags=tags[['userId','movieId','tag']]

In [19]:
len([*set(ratings.loc[ratings['userId']==3]['movieId'])])

39

In [20]:
ratings.shape

(100836, 3)

In [21]:
tags.shape

(3683, 3)

In [22]:
len([*set(movies['title'])])

9731

In [23]:
[*set(tags.loc[tags['userId']==7]['tag'])]

['way too long']

In [24]:
l=[]
for i in [*set(ratings['userId'])]:
    if len([*set(tags.loc[tags['userId']==i]['tag'])])==0:
        l.append(i)
l


[1,
 3,
 4,
 5,
 6,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 104,
 105,
 107,
 108,
 109,
 110,
 111,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 124,
 126,
 127,
 128,
 129,
 130,
 131,
 133,
 134,
 135,
 136,
 137,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 162,
 163,
 164,
 165,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 178,
 179,
 180,
 181,
 182,
 183,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,
 204,


In [25]:
len(l)

552

In [26]:
len([*set(ratings['userId'])])

610

In [27]:
ratings.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [118]:
new_df=pd.DataFrame(columns=[*set(ratings['movieId'])],index=[*set(ratings['userId'])])

In [49]:
[*set(ratings.loc[ratings['userId']==1]['movieId'])]

[1024,
 1,
 1025,
 3,
 2048,
 1029,
 6,
 1030,
 1031,
 1032,
 2054,
 2058,
 2571,
 527,
 1552,
 1042,
 2580,
 1049,
 2078,
 543,
 3617,
 1060,
 1573,
 2596,
 552,
 553,
 2090,
 1580,
 2093,
 2094,
 47,
 2096,
 1073,
 50,
 1587,
 2099,
 3639,
 1080,
 2105,
 2616,
 2617,
 1089,
 1090,
 2115,
 1092,
 2116,
 70,
 2628,
 1097,
 3147,
 590,
 592,
 593,
 1617,
 2640,
 596,
 1620,
 2641,
 2644,
 2648,
 1625,
 2137,
 2139,
 3671,
 2141,
 2654,
 2143,
 608,
 2657,
 3168,
 101,
 1127,
 3176,
 1644,
 110,
 1136,
 2161,
 3702,
 3703,
 2174,
 2692,
 648,
 1676,
 2700,
 2193,
 3729,
 661,
 151,
 2716,
 157,
 3740,
 3744,
 673,
 163,
 3243,
 1196,
 1197,
 1198,
 3247,
 3253,
 1206,
 1208,
 1210,
 1213,
 1214,
 1219,
 1220,
 1732,
 1222,
 1224,
 2761,
 1226,
 3273,
 2253,
 3793,
 216,
 1240,
 2268,
 733,
 223,
 736,
 2273,
 3809,
 231,
 1256,
 1258,
 235,
 2797,
 1265,
 1777,
 2291,
 1270,
 1275,
 1278,
 1793,
 1282,
 260,
 2826,
 1291,
 780,
 1804,
 1805,
 1298,
 2329,
 2338,
 804,
 296,
 2858,
 2353,

In [47]:
ratings.loc[ratings['userId']==1]['movieId'].shape

(232,)

In [51]:
ratings.loc[ratings['userId']==1]['movieId']

0         1
1         3
2         6
3        47
4        50
       ... 
227    3744
228    3793
229    3809
230    4006
231    5060
Name: movieId, Length: 232, dtype: int64

In [70]:
dic={}
for user in ratings['userId']:
    dic[user]=ratings.loc[ratings['userId']==1]['rating']
    

In [57]:
len([*set(ratings['userId'])])

610

In [127]:
for i in range(0,len(ratings)):
    new_df[int(ratings.iloc[i].userId)][int(ratings.iloc[i].movieId)]=ratings.iloc[i].rating

KeyError: 33

In [128]:
i

5031

In [107]:
ratings.iloc[100836-1]

userId        610.0
movieId    170875.0
rating          3.0
Name: 100835, dtype: float64

In [96]:
new_df[int(ratings.iloc[0].userId)][int(ratings.iloc[0].movieId)]

nan

In [142]:
ratings.loc[ratings['userId']==33]

,userId,movieId,rating
5031,33,1,3.0
5032,33,7,1.0
5033,33,11,2.0
5034,33,17,4.0
5035,33,21,4.0
...,...,...,...
5182,33,2690,4.0
5183,33,2724,3.0
5184,33,2762,4.0
5185,33,2875,4.0


In [110]:
new_df[int(ratings.iloc[33].userId)][int(ratings.iloc[33].movieId)]

4.0

In [122]:
int(ratings.iloc[33].userId)

1

In [135]:
new_df.iloc[33]

1         NaN
2         NaN
3         NaN
4         NaN
5         4.0
         ... 
163809    NaN
98279     NaN
32743     NaN
65514     NaN
98296     NaN
Name: 34, Length: 9724, dtype: object

In [143]:
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0
